# DeepESN <br/>
https://github.com/gallicch/DeepRC-TF

In [1]:
%matplotlib inline
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Input, Reshape, Dropout, Bidirectional, LSTM, GRU
from tensorflow.keras.preprocessing.text import Tokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, multilabel_confusion_matrix, plot_confusion_matrix, classification_report, coverage_error

from DeepRC import *

In [2]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [3]:
df = pd.read_csv("data/train.csv")
df['comment_text'] = df['comment_text'].map(lambda com : clean_text(com))

categories = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
num_classes = len(categories)
train, valid = train_test_split(df, random_state=42, test_size=0.33, shuffle=True)

In [4]:
df_test = pd.read_csv('data/test.csv')
df['comment_text'] = df['comment_text'].map(lambda com : clean_text(com))
cols = df_test.columns
label_cols = list(cols[2:])

test_labels_df = pd.read_csv('data/test_labels.csv')
df_test = df_test.merge(test_labels_df, on='id', how='left')
test_label_cols = list(df_test.columns[2:])
print('Null values: ', df_test.isnull().values.any()) #should not be any null sentences or labels
print('Same columns between train and test: ', label_cols == test_label_cols) #columns should be the same
print(f"Number of rows original: {df_test.shape[0]}")

df_test = df_test[~df_test[test_label_cols].eq(-1).any(axis=1)] #remove irrelevant rows/comments with -1 values
df_test['one_hot_labels'] = list(df_test[test_label_cols].values)
print(f"Number of rows after: {df_test.shape[0]}")
df_test.head()

Null values:  False
Same columns between train and test:  False
Number of rows original: 153164
Number of rows after: 63978


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,one_hot_labels
5,0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]"
7,000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]"
11,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]"
13,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]"
14,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]"


In [5]:
X_train = train.comment_text
y_train = train[categories]
X_valid = valid.comment_text
y_valid = valid[categories]
X_test = df_test.comment_text.values
y_test = df_test[categories].values

tokenizer = Tokenizer(num_words=500)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_matrix(X_train, mode='tfidf')
X_valid = tokenizer.texts_to_matrix(X_valid, mode='tfidf')
X_test = tokenizer.texts_to_matrix(X_test, mode='tfidf')

X_train = tf.expand_dims(X_train, axis=2)
X_valid = tf.expand_dims(X_valid, axis=2)

y_train = y_train.values
y_valid = y_valid.values

print('training set',X_train.shape,y_train.shape)
print('validation set',X_valid.shape,y_valid.shape)
print('test set',X_test.shape,y_test.shape)

training set (106912, 500, 1) (106912, 6)
validation set (52659, 500, 1) (52659, 6)
test set (63978, 500) (63978, 6)


In [15]:
reservoir_units = 100
batch_size = 32
lr = 0.001

inputs = Input(shape=(X_train.shape[1],1), name='inputs')
reservoir = SimpleDeepReservoirLayer(units=reservoir_units, layers=3, trainable=False, name='reservoir')(inputs)

#reservoir = Reshape((1,reservoir_units), input_shape=(reservoir_units,))(reservoir)
#readout = Bidirectional(GRU(10), merge_mode='sum')(reservoir)

readout = Dense(num_classes, activation='sigmoid', name='readout')(reservoir)

model = Model(inputs, readout, name='DeepESN')

model.compile(optimizer=Adam(learning_rate=lr), 
              loss='binary_crossentropy', 
              metrics=[
                    'accuracy',
                    'binary_accuracy', 
                    tf.keras.metrics.AUC(multi_label=True),
                    tf.keras.metrics.Precision(),
                    tf.keras.metrics.Recall()
              ])
model.summary()

Model: "DeepESN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 500, 1)]          0         
_________________________________________________________________
reservoir (SimpleDeepReservo (None, 100)               0         
_________________________________________________________________
readout (Dense)              (None, 6)                 606       
Total params: 606
Trainable params: 606
Non-trainable params: 0
_________________________________________________________________


In [16]:
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=2, verbose=1, validation_data=(X_valid,y_valid))

Epoch 1/2
3341/3341 [==============================] - 689s 206ms/step - loss: 0.1513 - accuracy: 0.9938 - binary_accuracy: 0.9605 - auc_4: 0.4996 - precision_4: 0.0751 - recall_4: 0.0081 - val_loss: 0.1430 - val_accuracy: 0.9940 - val_binary_accuracy: 0.9630 - val_auc_4: 0.5135 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00
Epoch 2/2
3341/3341 [==============================] - 682s 204ms/step - loss: 0.1432 - accuracy: 0.9938 - binary_accuracy: 0.9627 - auc_4: 0.5107 - precision_4: 0.0000e+00 - recall_4: 0.0000e+00 - val_loss: 0.1425 - val_accuracy: 0.9940 - val_binary_accuracy: 0.9630 - val_auc_4: 0.5200 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00


In [17]:
scores = model.evaluate(X_test, y_test, verbose=2)
print(f"Test {model.metrics_names}: {scores}")
scores = model.evaluate(X_valid, y_valid, verbose=2)
print(f"Validation {model.metrics_names}: {scores}")

2000/2000 - 264s - loss: 0.1442 - accuracy: 0.9976 - binary_accuracy: 0.9622 - auc_4: 0.5218 - precision_4: 0.0000e+00 - recall_4: 0.0000e+00
Test ['loss', 'accuracy', 'binary_accuracy', 'auc_4', 'precision_4', 'recall_4']: [0.14421361684799194, 0.9976085424423218, 0.962231457233429, 0.52182537317276, 0.0, 0.0]
1646/1646 - 223s - loss: 0.1425 - accuracy: 0.9940 - binary_accuracy: 0.9630 - auc_4: 0.5200 - precision_4: 0.0000e+00 - recall_4: 0.0000e+00
Validation ['loss', 'accuracy', 'binary_accuracy', 'auc_4', 'precision_4', 'recall_4']: [0.14247572422027588, 0.9940181374549866, 0.9629507660865784, 0.519951343536377, 0.0, 0.0]


In [ ]:
y_valid_pred = model.predict(X_valid)
y_test_pred = model.predict(X_test)

print(f"Validation scores:\n{classification_report(y_valid, y_valid_pred, target_names=categories)}")
print(f"Test scores:\n{classification_report(y_test, y_test_pred, target_names=categories)}")